In [117]:
!pip install pyspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install sparksql-magic

The system cannot find the path specified.
'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open 'spark-3.1.1-bin-hadoop3.2.tgz'


In [1]:
import pyspark
import os

print(os.path.dirname(pyspark.__file__))

C:\Users\navid\anaconda3\Lib\site-packages\pyspark


In [2]:
import os

# 👉 If you're on **Windows**, use your actual local Spark and Java paths
# Example paths (update these as per your install location)
os.environ["JAVA_HOME"] = "C:\\Program Files\\Java\\jdk-17"
os.environ["SPARK_HOME"] = "C:\\Users\\navid\\anaconda3\\Lib\\site-packages\\pyspark"

import findspark
findspark.init()

# import modules from pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession

from pyspark.sql import functions as F
from pyspark.sql.types import DecimalType
from pyspark.sql.functions import to_date

from pyspark.sql.functions import col, row_number, count, when, lit, first,regexp_extract
from pyspark.sql.window import Window

In [3]:
#spark = SparkSession.builder.appName("Kijiji Car Price Prediction App").config('spark.driver.memory', '4g').getOrCreate()
spark = SparkSession.builder \
.appName('Kijiji Car Price Prediction App') \
.master('local[*]') \
.config('spark.sql.execution.arrow.pyspark.enabled', True) \
.config('spark.sql.session.timeZone', 'UTC') \
.config('spark.driver.memory','32G') \
.config('spark.ui.showConsoleProgress', True) \
.config('spark.sql.repl.eagerEval.enabled', True) \
.getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
sqlContext = SQLContext(spark)
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

C:\Users\navid\anaconda3\Lib\site-packages\pyspark\sql\context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
# Download the Parquet file


# Read the Parquet file
kijiji = spark.read.parquet('data_2024_08_06.parquet')

# Show the DataFrame
kijiji.show()

+-----+---------+-------------+-----------+-----------+--------------------+-----------+---------+---------+----------+-----------------+--------------------+------------+-------------+---------------+-----------------+-------------+-----------+-----------------+-------------+---------+-----------+-----------+---------+---+-----+--------------------+---------------+---------------+-----------------+--------------+------------+
|index|  stock_s|vehicletype_s|bluetooth_b|forsaleby_s|      carprooflink_s|noofdoors_s|carmake_s|sunroof_b|carcolor_s|carmileageinkms_i|           cartrim_s|drivetrain_s|alloywheels_b|     carmodel_s|cartransmission_s|carbodytype_s|navsystem_b|pushbuttonstart_b|carfueltype_s|caryear_i|noofseats_s|date_posted|      amt|cur|    t|                 url|parkingassist_b|cruisecontrol_b|airconditioning_b|trailerhitch_b|cpoprogram_s|
+-----+---------+-------------+-----------+-----------+--------------------+-----------+---------+---------+----------+-----------------+-

In [5]:
#summary df before clean
kijiji.describe().toPandas()

,summary,index,stock_s,vehicletype_s,bluetooth_b,forsaleby_s,carprooflink_s,noofdoors_s,carmake_s,sunroof_b,...,date_posted,amt,cur,t,url,parkingassist_b,cruisecontrol_b,airconditioning_b,trailerhitch_b,cpoprogram_s
0,count,185289,122416,183104,108830,183104,50840,125589,183104,51180,...,183034,175929,183114,183114,185289,48639,17123,18416,7103,4074
1,mean,3543.66871751696,Infinity,None,0.9996974510248846,None,None,3.8789995852872683,None,0.9996161596775741,...,None,3384486.672652036,None,None,None,0.9990582030514221,0.9998826015496596,0.9998909071074019,0.9996751137102015,None
2,stddev,16062.531945844603,NaN,None,0.017391961304308246,None,None,0.5734729882195206,None,0.01958902273663544,...,None,3981829.3433599058,None,None,None,0.030677145070912175,0.010834738517281011,0.010444469428546195,0.018023138063986022,None
3,min,0,3204004-KZ,damaged,,delr,,,acura,,...,2020-12-03,30.0,CAD,CONTACT,https://www.kijiji.ca/v-cars-trucks/100-mile-h...,,,,,acura
4,max,266630,À DÉTERMINER,used,true,ownr,https://vhr.carfax.ca/main?id=zTejfQoNp+0dTrnI...,others,volvo,true,...,2024-08-06,8.008135E8,CAD,SWAP_TRADE,https://www.kijiji.ca/v-cars-trucks/yellowknif...,true,1,1,true,volkwagen


In [6]:
# remove the unneeded columns
kijiji = kijiji.drop(*["stock_s", "t", "url", "cur", "date_posted"])

In [7]:
# check if any columns has NULL before cleaning up
from pyspark.sql.functions import col, when

for c in kijiji.columns:
    null_count = kijiji.where(col(c).isNull()).count()
    if null_count > 0:
        print(f"The column '{c}' has {null_count} NULL values.")

The column 'vehicletype_s' has 2185 NULL values.
The column 'bluetooth_b' has 76459 NULL values.
The column 'forsaleby_s' has 2185 NULL values.
The column 'carprooflink_s' has 134449 NULL values.
The column 'noofdoors_s' has 59700 NULL values.
The column 'carmake_s' has 2185 NULL values.
The column 'sunroof_b' has 134109 NULL values.
The column 'carcolor_s' has 27964 NULL values.
The column 'carmileageinkms_i' has 2185 NULL values.
The column 'cartrim_s' has 30090 NULL values.
The column 'drivetrain_s' has 29379 NULL values.
The column 'alloywheels_b' has 124364 NULL values.
The column 'carmodel_s' has 2185 NULL values.
The column 'cartransmission_s' has 25531 NULL values.
The column 'carbodytype_s' has 2185 NULL values.
The column 'navsystem_b' has 139517 NULL values.
The column 'pushbuttonstart_b' has 109500 NULL values.
The column 'carfueltype_s' has 27029 NULL values.
The column 'caryear_i' has 2185 NULL values.
The column 'noofseats_s' has 67561 NULL values.
The column 'amt' has 9

In [8]:
# remove rows with NULL
kijiji = kijiji.filter(kijiji.caryear_i.isNotNull())
kijiji = kijiji.filter(kijiji.vehicletype_s.isNotNull())
kijiji = kijiji.filter(kijiji.carmake_s.isNotNull())
kijiji = kijiji.filter(kijiji.forsaleby_s.isNotNull())
kijiji = kijiji.filter(kijiji.carmileageinkms_i.isNotNull())
kijiji = kijiji.filter(kijiji.carmodel_s.isNotNull())
kijiji = kijiji.filter(kijiji.carbodytype_s.isNotNull())

# convert the amt column from cents to dollars
kijiji = kijiji.filter(kijiji.amt.isNotNull())
kijiji = kijiji.withColumn("amt", col("amt") / 100)


# column carproof
url_pattern = r'(http|https)://[a-zA-Z0-9\\./]+'

# Check if the carprooflink_s column is a valid hyperlink
kijiji = kijiji.withColumn("carprooflink_s", when(regexp_extract(col("carprooflink_s"), url_pattern, 0) != "", True).otherwise(False))

In [9]:
kijiji.columns

['index',
 'vehicletype_s',
 'bluetooth_b',
 'forsaleby_s',
 'carprooflink_s',
 'noofdoors_s',
 'carmake_s',
 'sunroof_b',
 'carcolor_s',
 'carmileageinkms_i',
 'cartrim_s',
 'drivetrain_s',
 'alloywheels_b',
 'carmodel_s',
 'cartransmission_s',
 'carbodytype_s',
 'navsystem_b',
 'pushbuttonstart_b',
 'carfueltype_s',
 'caryear_i',
 'noofseats_s',
 'amt',
 'parkingassist_b',
 'cruisecontrol_b',
 'airconditioning_b',
 'trailerhitch_b',
 'cpoprogram_s']

* airconditioning_b: A boolean indicating if the car has air conditioning.
* alloywheels_b: A boolean indicating if the car has alloy wheels.
* amt: The amount or price of the car.
* bluetooth_b: A boolean indicating if the car has Bluetooth.
* carbodytype_s: The body type of the car (e.g., sedan, suvcrossover).
* carcolor_s: The color of the car.
* carfueltype_s: The fuel type of the car (e.g., gas).
* carmake_s: The make of the car (e.g., kia, honda).
* carmileageinkms_i: The mileage of the car in kilometers.
* carmodel_s: The model of the car (e.g., forte, hrv).
* carprooflink_s: The link to the CarProof report for the car.
* cartransmission_s: The transmission of the car (e.g., 2).
* cartrim_s: The trim of the car.
* caryear_i: The year of the car.
* cpoprogram_s: The certified pre-owned program of the car.
* cruisecontrol_b: A boolean indicating if the car has cruise control.
* cur: The currency of the price (e.g., CAD).
* date_posted: The date the car was posted for sale.
* drivetrain_s: The drivetrain of the car (e.g., fwd, awd).
* forsaleby_s: Who the car is for sale by (e.g., delr, ownr).
* navsystem_b: A boolean indicating if the car has a navigation system.
* noofdoors_s: The number of doors on the car.
* noofseats_s: The number of seats in the car.
* parkingassist_b: A boolean indicating if the car has parking assist.
* pushbuttonstart_b: A boolean indicating if the car has push button start.
* stock_s: The stock number of the car.
* sunroof_b: A boolean indicating if the car has a sunroof.
* t: The type of listing (e.g., FIXED, CONTACT).
* trailerhitch_b: A boolean indicating if the car has a trailer hitch.
* url: The URL of the listing.
* vehicletype_s: The type of vehicle (e.g., used, new).

In [11]:
# before cleaning
print("noofdoors_s - before cleaing")
kijiji.groupBy("noofdoors_s").count().show()


# Assuming your dataset is in a DataFrame called df
# Group by carmake_s, carmodel_s, and carbodytype_s and calculate the mode for noofdoors_s
window_spec = Window.partitionBy("carmake_s", "carmodel_s", "carbodytype_s")

# Calculate the mode for noofdoors_s within each group
mode_noofdoors = kijiji.groupBy("carmake_s", "carmodel_s", "carbodytype_s", "noofdoors_s") \
                   .agg(count("noofdoors_s").alias("count")) \
                   .withColumn("rank", row_number().over(Window.partitionBy("carmake_s", "carmodel_s", "carbodytype_s").orderBy(col("count").desc()))) \
                   .filter(col("rank") == 1) \
                   .select("carmake_s", "carmodel_s", "carbodytype_s", "noofdoors_s") \
                   .withColumnRenamed("noofdoors_s", "mode_noofdoors")

print("noofdoors_s - with carmake_s, carmodel_s, carbodytype_s, noofdoors_s")
print(mode_noofdoors.toPandas())

#mode_noofdoors = mode_noofdoors.withColumn('carfueltype_s', when(col('carbodytype_s') == 'htchbck', 'Not specified').otherwise(col('carfueltype_s')))
#mode_noofdoors.groupBy("mode_noofdoors").count().show()

print("--------------------------")
print("noofdoors_s - with carmake_s, carmodel_s, carbodytype_s, noofdoors_s, but have NULL value")
mode_noofdoors.filter(col("mode_noofdoors").isNull()).groupBy("carbodytype_s").count().show()
print(mode_noofdoors.filter(col("mode_noofdoors").isNull()).count())
print("--------------------------")

##mode_noofdoors.filter((col("carbodytype_s") == "htchbck") & (col("mode_noofdoors").isNull())).groupBy("carbodytype_s").count().show()
##mode_noofdoors.filter((col("carbodytype_s") == "vanminicomma") & (col("mode_noofdoors").isNotNull())).toPandas()

# if decide to remove them,
mode_noofdoors = mode_noofdoors.filter(col("mode_noofdoors").isNotNull())


# Join the mode_noofdoors back to the original DataFrame
kijiji = kijiji.join(mode_noofdoors, on=["carmake_s", "carmodel_s", "carbodytype_s"], how="left")

# Fill NULL values in noofdoors_s with the mode
kijiji = kijiji.withColumn("noofdoors_s", when(col("noofdoors_s").isNull(), col("mode_noofdoors")).otherwise(col("noofdoors_s")))

# Drop the temporary mode_noofdoors column
kijiji = kijiji.drop("mode_noofdoors")

# noofdoors_s - before cleaing
print("noofdoors_s - after cleaing")
kijiji.groupBy("noofdoors_s").count().show()

noofdoors_s - before cleaing
+-----------+------+
|noofdoors_s| count|
+-----------+------+
|          3|   256|
|       NULL| 55020|
|          5|  4202|
|     others|    89|
|          4|106925|
|           |    94|
|          2|  9334|
+-----------+------+

noofdoors_s - with carmake_s, carmodel_s, carbodytype_s, noofdoors_s
     carmake_s carmodel_s carbodytype_s mode_noofdoors
0        acura         cl          conv           None
1        acura         cl          coup              2
2        acura         cl       htchbck           None
3        acura         cl    othrbdytyp           None
4        acura         cl         sedan              4
...        ...        ...           ...            ...
3022     volvo       xc90    othrbdytyp              4
3023     volvo       xc90         sedan              4
3024     volvo       xc90  suvcrossover              4
3025     volvo       xc90         wagon              4
3026     volvo       xccc         sedan           None

[3027 row

In [12]:
# before cleaning
print("noofseats_s - before cleaning")
kijiji.groupBy("noofseats_s").count().show()

# handling rows
#kijiji = kijiji.withColumn('noofseats_s', F.when(F.col('noofseats_s').isNull(), 0).otherwise(F.col('noofseats_s').cast("string")))
#kijiji = kijiji.withColumn('noofseats_s', when(col('noofseats_s') == '', 'Not specified').otherwise(col('noofseats_s')))

# Assuming your dataset is in a DataFrame called df
# Group by carmake_s, carmodel_s, and carbodytype_s and calculate the mode for noofseats_s
window_spec = Window.partitionBy("carmake_s", "carmodel_s", "carbodytype_s")

# Calculate the mode for noofseats_s within each group
mode_noseats = kijiji.groupBy("carmake_s", "carmodel_s", "carbodytype_s", "noofseats_s") \
                   .agg(count("noofseats_s").alias("count")) \
                   .withColumn("rank", row_number().over(Window.partitionBy("carmake_s", "carmodel_s", "carbodytype_s").orderBy(col("count").desc()))) \
                   .filter(col("rank") == 1) \
                   .select("carmake_s", "carmodel_s", "carbodytype_s", "noofseats_s") \
                   .withColumnRenamed("noofseats_s", "mode_noseats")

print("noofseats_s - with carmake_s, carmodel_s, carbodytype_s, noofseats_s")
print(mode_noseats.toPandas())

#mode_noseats = mode_noseats.withColumn('carfueltype_s', when(col('carbodytype_s') == 'htchbck', 'Not specified').otherwise(col('carfueltype_s')))
#mode_noseats.groupBy("mode_noseats").count().show()

print("--------------------------")
print("noofseats_s - with carmake_s, carmodel_s, carbodytype_s, noofseats_s, but have NULL value")
mode_noseats.filter(col("mode_noseats").isNull()).groupBy("carbodytype_s").count().show()
print(mode_noseats.filter(col("mode_noseats").isNull()).count())
print("--------------------------")

##mode_noseats.filter((col("carbodytype_s") == "htchbck") & (col("mode_noseats").isNull())).groupBy("carbodytype_s").count().show()
##mode_noseats.filter((col("carbodytype_s") == "vanminicomma") & (col("mode_noseats").isNotNull())).toPandas()

# if decide to remove them,
mode_noseats = mode_noseats.filter(col("mode_noseats").isNotNull())


# Join the mode_noseats back to the original DataFrame
kijiji = kijiji.join(mode_noseats, on=["carmake_s", "carmodel_s", "carbodytype_s"], how="left")

# Fill NULL values in noofseats_s with the mode
kijiji = kijiji.withColumn("noofseats_s", when(col("noofseats_s").isNull(), col("mode_noseats")).otherwise(col("noofseats_s")))

# Drop the temporary mode_noofdoors column
kijiji = kijiji.drop("mode_noseats")


# before cleaning
print("noofseats_s - after cleaning")
kijiji.groupBy("noofseats_s").count().show()


noofseats_s - before cleaning
+-----------+-----+
|noofseats_s|count|
+-----------+-----+
|          7| 9497|
|         15|    5|
|          3|  646|
|          8| 1605|
|       NULL|62656|
|          5|85016|
|         18|    3|
|      other|  384|
|          6| 6950|
|          9|   59|
|          1|  349|
|          4| 5568|
|         12|   14|
|           |  108|
|         13|    1|
|          2| 3059|
+-----------+-----+

noofseats_s - with carmake_s, carmodel_s, carbodytype_s, noofseats_s
     carmake_s carmodel_s carbodytype_s mode_noseats
0        acura         cl          conv         None
1        acura         cl          coup            4
2        acura         cl       htchbck         None
3        acura         cl    othrbdytyp         None
4        acura         cl         sedan            5
...        ...        ...           ...          ...
3022     volvo       xc90    othrbdytyp            7
3023     volvo       xc90         sedan            7
3024     volvo       xc

In [13]:
# we decided to remove those NULL rows left,
kijiji = kijiji.filter(col("noofseats_s").isNotNull())
kijiji = kijiji.filter(col("noofdoors_s").isNotNull())

In [14]:
# clean up the data
kijiji = kijiji.withColumn('airconditioning_b', F.when(F.col('airconditioning_b').isNull(), False).otherwise(True))
kijiji = kijiji.withColumn('alloywheels_b', F.when(F.col('alloywheels_b').isNull(), False).otherwise(True))
kijiji = kijiji.withColumn('bluetooth_b', F.when(F.col('bluetooth_b').isNull(), False).otherwise(True))
kijiji = kijiji.withColumn('sunroof_b', F.when(F.col('sunroof_b').isNull(), False).otherwise(True))
kijiji = kijiji.withColumn('parkingassist_b', F.when(F.col('parkingassist_b').isNull(), False).otherwise(True))
kijiji = kijiji.withColumn('cruisecontrol_b', F.when(F.col('cruisecontrol_b').isNull(), False).otherwise(True))
kijiji = kijiji.withColumn('pushbuttonstart_b', F.when(F.col('pushbuttonstart_b').isNull(), False).otherwise(True))
kijiji = kijiji.withColumn('trailerhitch_b', F.when(F.col('trailerhitch_b').isNull(), False).otherwise(True))
kijiji = kijiji.withColumn('navsystem_b', F.when(F.col('navsystem_b').isNull(), False).otherwise(True))

kijiji = kijiji.withColumn('carmileageinkms_i', F.when(F.col('carmileageinkms_i').isNull(), 0).otherwise(F.col('carmileageinkms_i').cast("int")))
kijiji = kijiji.withColumn('carbodytype_s', F.when(F.col('carbodytype_s').isNull(), "Not Specified").otherwise(F.col('carbodytype_s')))
kijiji = kijiji.withColumn('carcolor_s', F.when(F.col('carcolor_s').isNull(), "Not Specified").otherwise(F.col('carbodytype_s')))

kijiji = kijiji.withColumn('carfueltype_s', F.when(F.col('carfueltype_s').isNull(), "Not Specified").otherwise(F.col('carfueltype_s')))
kijiji = kijiji.withColumn('carfueltype_s', when(col('carfueltype_s') == 'others', 'Not specified').otherwise(col('carfueltype_s')))

kijiji = kijiji.withColumn('carmake_s', F.when(F.col('carmake_s').isNull(), "Not Specified").otherwise(F.col('carmake_s')))
kijiji = kijiji.withColumn('carmodel_s', F.when(F.col('carmodel_s').isNull(), "Not Specified").otherwise(F.col('carmodel_s')))
kijiji = kijiji.withColumn('cartransmission_s', F.when(F.col('cartransmission_s').isNull(), "Not Specified").otherwise(F.col('cartransmission_s')))
kijiji = kijiji.withColumn('vehicletype_s', F.when(F.col('vehicletype_s').isNull(), "Not Specified").otherwise(F.col('vehicletype_s')))

kijiji = kijiji.withColumn('cartrim_s', F.when(F.col('cartrim_s').isNull(), "Not Specified").otherwise(F.col('cartrim_s')))
kijiji = kijiji.withColumn('cartrim_s', when(col('cartrim_s') == '', 'Not specified').otherwise(col('cartrim_s')))

kijiji = kijiji.withColumn('cpoprogram_s', F.when(F.col('cpoprogram_s').isNull(), "Not Specified").otherwise(F.col('cpoprogram_s')))
kijiji = kijiji.withColumn('forsaleby_s', F.when(F.col('forsaleby_s').isNull(), "Not Specified").otherwise(F.col('forsaleby_s')))
kijiji = kijiji.withColumn('drivetrain_s', F.when(F.col('drivetrain_s').isNull(), "Not Specified").otherwise(F.col('drivetrain_s')))
kijiji = kijiji.withColumn('drivetrain_s', when(col('drivetrain_s') == '', 'Not specified').otherwise(col('drivetrain_s')))

In [15]:
# check if any columns has NULL, after cleaning
from pyspark.sql.functions import col, when

for c in kijiji.columns:
    null_count = kijiji.where(col(c).isNull()).count()
    if null_count > 0:
        print(f"The column '{c}' has {null_count} NULL values.")

In [16]:
# convert to Pandas to read
df = kijiji.toPandas()
df

,carmake_s,carmodel_s,carbodytype_s,index,vehicletype_s,bluetooth_b,forsaleby_s,carprooflink_s,noofdoors_s,sunroof_b,...,pushbuttonstart_b,carfueltype_s,caryear_i,noofseats_s,amt,parkingassist_b,cruisecontrol_b,airconditioning_b,trailerhitch_b,cpoprogram_s
0,dodge,ram2500,pickuptruck,80,used,False,ownr,False,4,False,...,False,Not Specified,2003.0,6,6500.0,False,False,False,False,Not Specified
1,rollsroyce,cullinan,suvcrossover,243,used,False,delr,False,4,False,...,False,gas,2022.0,5,439995.0,False,False,False,False,Not Specified
2,dodge,ram2500,pickuptruck,1075,used,False,ownr,False,4,False,...,False,Not Specified,1998.0,6,7500.0,False,False,False,False,Not Specified
3,dodge,ram2500,pickuptruck,1472,used,False,ownr,False,4,True,...,False,diesel,2006.0,6,11500.0,False,True,True,True,Not Specified
4,dodge,ram2500,pickuptruck,4072,used,False,ownr,False,4,False,...,False,Not Specified,2008.0,6,6350.0,False,False,False,False,Not Specified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174697,volkwagen,beetle,conv,385,used,True,ownr,False,2,False,...,False,gas,2018.0,4,25999.0,False,False,False,False,Not Specified
174698,volkwagen,beetle,conv,151,used,True,delr,False,2,False,...,True,gas,2019.0,4,32500.0,True,False,False,False,Not Specified
174699,volkwagen,beetle,conv,395,used,False,delr,True,2,False,...,False,gas,2019.0,4,30888.0,False,False,False,False,volkwagen
174700,volkwagen,beetle,conv,559,used,True,delr,False,2,True,...,True,other,2019.0,4,11995.0,False,False,False,False,Not Specified


In [17]:
!pip show pyspark

Name: pyspark
Version: 4.0.0
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: C:\Users\navid\anaconda3\Lib\site-packages
Requires: py4j
Required-by: sparksql-magic


In [18]:
!spark-submit --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 4.0.0
      /_/
                        
Using Scala version 2.13.16, Java HotSpot(TM) 64-Bit Server VM, 17.0.12
Branch HEAD
Compiled by user wenchen on 2025-05-19T07:58:03Z
Revision fa33ea000a0bda9e5a3fa1af98e8e85b8cc5e4d4
Url https://github.com/apache/spark
Type --help for more information.


In [19]:
# get describe after clean
kijiji.describe().toPandas()


,summary,carmake_s,carmodel_s,carbodytype_s,index,vehicletype_s,forsaleby_s,noofdoors_s,carcolor_s,carmileageinkms_i,cartrim_s,drivetrain_s,cartransmission_s,carfueltype_s,caryear_i,noofseats_s,amt,cpoprogram_s
0,count,174702,174702,174702,174702,174702,174702,174702,174702,174702,174702,174702,174702,174702,174702,174702,174702,174702
1,mean,None,1640.3317721836581,None,3591.5154434408305,None,None,3.8601573738172608,None,102477.99555242642,2461.9508979591837,None,1.9872320693843963,None,2017.460733134137,5.082364424539948,33892.199071561765,None
2,stddev,None,987.0616728318198,None,16157.359069294582,None,None,0.584826300174771,None,119123.82819630315,19999.447533580627,None,0.32803085499055756,None,5.940824789132284,0.9573345961611311,39733.82930779103,None
3,min,acura,124_spider,conv,0,damaged,delr,,Not Specified,0,,4x4,,,1933.0,,0.3,Not Specified
4,max,volvo,zseries,wagon,266630,used,ownr,others,wagon,10000000,⁷,rwd,Not Specified,other,2025.0,other,8008135.0,volkwagen


In [20]:
# look into vehicle type
kijiji.groupBy("vehicletype_s").count().show()

+-------------+------+
|vehicletype_s| count|
+-------------+------+
|         used|139734|
|          new| 33979|
|        lease|   379|
|      salvage|   190|
|      damaged|   420|
+-------------+------+



In [21]:
# for determine if it's good buy, the vehicletype_s must be not damage
kijiji = kijiji.filter(kijiji.vehicletype_s != "damaged")
kijiji = kijiji.filter(kijiji.vehicletype_s != "salvage")
kijiji = kijiji.filter(kijiji.vehicletype_s != "Not Specified")

In [22]:
# Feature engineer: Combine 'carmake_s' and 'carmodel_s' into a new column 'make_model'
kijiji = kijiji.withColumn("make_model", concat_ws("_", col("carmake_s"), col("carmodel_s")))

# Drop the original columns 'carmodel_s', and 'cartrim_s', as this leads to computing resource issues that our group couldn't manage given our current resources. Make and trim levels just add to too many dummy (binary) variables.
kijiji = kijiji.drop("carmodel_s", "cartrim_s")

# Show the updated DataFrame (optional)
kijiji.show()

+----------+-------------+------+-------------+-----------+-----------+--------------+-----------+---------+-------------+-----------------+-------------+-------------+-----------------+-----------+-----------------+-------------+---------+-----------+--------+---------------+---------------+-----------------+--------------+-------------+-------------------+
| carmake_s|carbodytype_s| index|vehicletype_s|bluetooth_b|forsaleby_s|carprooflink_s|noofdoors_s|sunroof_b|   carcolor_s|carmileageinkms_i| drivetrain_s|alloywheels_b|cartransmission_s|navsystem_b|pushbuttonstart_b|carfueltype_s|caryear_i|noofseats_s|     amt|parkingassist_b|cruisecontrol_b|airconditioning_b|trailerhitch_b| cpoprogram_s|         make_model|
+----------+-------------+------+-------------+-----------+-----------+--------------+-----------+---------+-------------+-----------------+-------------+-------------+-----------------+-----------+-----------------+-------------+---------+-----------+--------+---------------+-

In [23]:
columns_to_check = ['noofdoors_s', 'cartransmission_s', 'carfueltype_s', 'noofseats_s']

# Drop rows with null or empty values in the specified columns
kijiji_cleaned = kijiji.filter(
    (col('noofdoors_s').isNotNull()) & (col('noofdoors_s') != "") &
    (col('cartransmission_s').isNotNull()) & (col('cartransmission_s') != "") &
    (col('carfueltype_s').isNotNull()) & (col('carfueltype_s') != "") &
    (col('noofseats_s').isNotNull()) & (col('noofseats_s') != "")
)

# Verify the rows are dropped
kijiji_cleaned.count()  # This should be less than the original row count

173959

In [24]:
#ML Fitting
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
import pickle

# Define categorical, binary, and numerical columns
categorical_cols = ['vehicletype_s', 'forsaleby_s', 'noofdoors_s', 'carcolor_s', 'make_model',
                    'drivetrain_s', 'cartransmission_s', 'carbodytype_s', 'carfueltype_s',
                    'noofseats_s', 'cpoprogram_s']

binary_cols = ['bluetooth_b', 'sunroof_b', 'alloywheels_b', 'navsystem_b', 'pushbuttonstart_b',
               'parkingassist_b', 'cruisecontrol_b', 'airconditioning_b', 'trailerhitch_b',
               'carprooflink_s']

numerical_cols = ['carmileageinkms_i', 'caryear_i']

target_col = 'amt'

# Step 1: Index categorical columns
indexers = [StringIndexer(inputCol=col, outputCol=col+"_index") for col in categorical_cols]

# Step 2: OneHotEncode the indexed columns
encoders = [OneHotEncoder(inputCols=[col+"_index"], outputCols=[col+"_vec"]) for col in categorical_cols]

# Step 3: Assemble all features into a single vector
assembler = VectorAssembler(inputCols=[col+"_vec" for col in categorical_cols] + binary_cols + numerical_cols,
                            outputCol="features")

# Step 4: Create a pipeline to apply the transformations
pipeline = Pipeline(stages=indexers + encoders + [assembler])

# Step 5: Fit the pipeline and transform the data
kijiji_prepared = pipeline.fit(kijiji_cleaned).transform(kijiji_cleaned)

In [25]:
# Split the data into training and testing sets
train_data, test_data = kijiji_prepared.randomSplit([0.8, 0.2], seed=1234)

In [26]:
from pyspark.ml.regression import GBTRegressor

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql.functions import col, abs

# Gradient-Boosted Tree Regressor
gbt = GBTRegressor(featuresCol="features", labelCol=target_col, maxIter=100)

gbt_paramGrid = (ParamGridBuilder()
                 .addGrid(gbt.maxDepth, [2, 4])
                 .addGrid(gbt.maxBins, [32, 64])
                 .build())
evaluator = RegressionEvaluator(labelCol=target_col, predictionCol="prediction", metricName="rmse")

gbt_cv = CrossValidator(estimator=gbt, estimatorParamMaps=gbt_paramGrid, evaluator=evaluator, numFolds=3)
gbt_cvModel = gbt_cv.fit(train_data)
gbt_predictions = gbt_cvModel.transform(test_data)

In [ ]:
from pyspark.ml.regression import RandomForestRegressor

# Random Forest Regressor
rf = RandomForestRegressor(featuresCol="features", labelCol=target_col)
rf_paramGrid = (ParamGridBuilder()
                .addGrid(rf.numTrees, [50, 100])
                .addGrid(rf.maxDepth, [5, 10])
                .addGrid(rf.maxBins, [32, 64])
                .build())
rf_cv = CrossValidator(estimator=rf, estimatorParamMaps=rf_paramGrid, evaluator=evaluator, numFolds=3)
rf_cvModel = rf_cv.fit(train_data)
rf_predictions = rf_cvModel.transform(test_data)

In [ ]:
print("The Best Hyper Parameters Gradient-Boosted Tree Regressor:\n--------------------")
print(gbt_cvModel.bestModel)
print(gbt_cvModel.bestModel.extractParamMap())
print("")
print("")
print("The Best Hyper Parameters Random Forest Regressor:\n--------------------")
print(rf_cvModel.bestModel)
print(rf_cvModel.bestModel.extractParamMap())

In [ ]:
# Evaluator
def evaluate_model(predictions):
    # RMSE
    rmse = evaluator.evaluate(predictions)
    print(f"Root Mean Squared Error (RMSE): {rmse}")

    # MAE
    mae_evaluator = RegressionEvaluator(labelCol=target_col, predictionCol="prediction", metricName="mae")
    mae = mae_evaluator.evaluate(predictions)
    print(f"Mean Absolute Error (MAE): {mae}")

    # MAPE
    predictions = predictions.withColumn("ape", abs((col(target_col) - col("prediction")) / col(target_col)))
    mape = predictions.select("ape").agg({"ape": "avg"}).collect()[0][0]
    print(f"Mean Absolute Percentage Error (MAPE): {mape}")

# Evaluate GBT Model
print("Evaluating Gradient-Boosted Tree Regressor:")
evaluate_model(gbt_predictions)

# Evaluate Random Forest Model
print("\nEvaluating Random Forest Regressor:")
evaluate_model(rf_predictions)


In [ ]:
gbt_results = gbt_predictions.select("amt", "prediction").toPandas()
gbt_results

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.scatter(gbt_results['amt'], gbt_results['prediction'], color='blue', label='Data points')
#a, b = np.polyfit(gbt_results['amt'], gbt_results['prediction'], 1)
#plt.plot(gbt_results['amt'], a * gbt_results['amt'] + b, color='red', label='Line of best fit')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs. Predicted Values')
#plt.legend()
plt.show()

In [ ]:
plt.scatter(gbt_results['amt'], gbt_results['prediction'], color='blue', label='Data points')
plt.xlim(0, 500000)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.suptitle('Actual vs. Gradient Boosted Tree Regressor Model Predicted Values')
plt.title('*Limited to cars under 1/2 million CAD')
plt.show()

In [ ]:
gbt_results['percenatage_difference'] = (gbt_results['amt'] - gbt_results['prediction']) / gbt_results['amt']
gbt_results

In [ ]:
bins = np.linspace(-1, 1, 50)
plt.hist(gbt_results['percenatage_difference'], bins=bins, color='blue', edgecolor='black')
# Add labels and title
plt.xlabel('Percentage Difference - (Actual Listed Price - Predicted Price)/Actual Listed Price')
plt.ylabel('Frequency')
plt.title('Gradient Boosted Tree Regressor - Predicted Percentage Difference Distribution');